### Система мягких сфер
$$
    \Phi(r) = \varepsilon\left(\frac{\sigma}{r}\right)^n, \; n\in(3,+\infty)\\
    F=-T\ln{Z}
    Z_n=\int\frac{d^{3N}pd^{3N}q}{N!h^{3N}}\rho e^{-\beta H(p,q)}=
    \frac{1}{N!h^{3N}}\int e^{-\sigma^n\frac{\varepsilon}{kT}\sum r_{ij}^{-n}}d\overline{r}^N=\\=
    \frac{V^N}{N!h^{3N}}
    \int\exp\left(-\tilde{\rho}^{\frac{n}{3}}\frac{\varepsilon}{kT}\sum s_{ij}^{-n}\right)d\overline{s}^N=
    \frac{V^N}{N!h^{3N}}
    \int\exp{\left[-\left(\frac{6\zeta}{\pi}\right)^{\frac{3}{n}}\sum s_{ij}^{-n}\right]}d\overline{s}^N,
$$

где 
$$
    \overline{s}=\overline{r}\left(\frac{N}{V}\right)^{\frac{1}{3}}, \;
    \Lambda = \frac{h^2}{2\pi mkT}, \; 
    \tilde{\rho} = \frac{N\sigma^3}{V}
$$

Таким образом, свободная энергия зависит только от парамметра $\zeta=\frac{\pi}{6}\left(\frac{\varepsilon}{kT}\right)^{\frac{3}{n}}\rho\sigma^3, \; \rho=\frac{N}{V}$.

Таким образом, для одного и того же $\zeta$ величины $\frac{pV}{NkT}$, $\frac{E}{NkT}$, $D\left(\frac{N}{V}\right)^{\frac{1}{3}}\left(\frac{m}{kT}\right)^{\frac{1}{2}}$ и другиие одинаковы.

Выбираем $n=6$, $\zeta=1.18$ (соответствует точке плавления). Гранецентрированная решётка ($N=4s^3=256$).

Также посмотрим систему при $\zeta=1.5$ (кристал) и $\zeta=0.8$ (жидкость). Нужно применить алгоритм Метрополиса с периодичесскими граничными условиями.

### Теорема вириала для систеемы мягких сфер.

Вириал $\sum \overline{F}_i\overline{r}_i$.

Теорема: $2\left<T\right>=-\sum\limits_{i=1}^N\overline{F}_i\overline{r}_i$

Для однородных потенциалов $2\left<T\right>=n\left<U\right>$.

Можно показать, что $\frac{1}{3}\left<\sum \overline{F}_i\overline{r}_i\right>=-k$

In [2]:
import numpy as np
from numpy import sqrt

In [19]:
a = 1 #lattice constant

#inicial cube of face-centered lattice
cube = [(0, 0, 0), 
        (a, 0, 0), 
        (0, a, 0), 
        (a, a, 0), 
        (0, 0, a), 
        (a, 0, a), 
        (0, a, a), 
        (a, a, a),#end of usual cubic lattice
        (a/2, a/2, 0),
        (a/2, 0, a/2),
        (0, a/2, a/2),
        (a/2, a/2, a),
        (a/2, a, a/2),
        (a, a/2, a/2)]#end of face-centered lattice

#Calculates distances between two vectors
def distance(A, B):
    return sqrt((A[0] - B[0])**2 + (A[1] - B[1])**2 + (A[2] - B[2])**2)

In [20]:
#This function shifts the initial cube by given (dx, dy, dz) directions, where dx, dy, dz are integers
def generate_cube(dx, dy, dz, a = a, initial_cube = cube):#in units of a
    def add(vec_1, vec_2):
        return (vec_1[0] + vec_2[0], vec_1[1] + vec_2[1], vec_1[2] + vec_2[2])
    
    d_vec = (dx * a, dy * a, dz * a)

    return [add(i, d_vec) for i in cube]


In [41]:
#Create cubic cell
N = 4 #number of cubes in a row

set_of_coordinates = set()
for x in range(N):
    for y in range(N):
        for z in range(N):
            new_cube = generate_cube(x, y, z)
            for coord in new_cube:
                set_of_coordinates.add(coord)


coordinates_of_all_atoms_in_lattice = list(set_of_coordinates)

print("Number of atoms in given ")

365

In [48]:
list(set_of_coordinates)

[(3, 0, 3),
 (1, 4, 4),
 (3, 0.5, 3.5),
 (3.5, 0.5, 3),
 (3.5, 0, 3.5),
 (0, 2, 1),
 (0.5, 2, 1.5),
 (0.5, 2.5, 1),
 (0, 2.5, 1.5),
 (3, 1, 4),
 (3.5, 1.5, 4),
 (1, 2, 0),
 (1, 2.5, 0.5),
 (1.5, 2, 0.5),
 (1.5, 2.5, 0),
 (4, 1, 1),
 (4, 1.5, 1.5),
 (0, 4, 2),
 (0.5, 4, 2.5),
 (0, 1, 0),
 (0.5, 1.5, 0),
 (0, 1.5, 0.5),
 (0.5, 1, 0.5),
 (1, 0, 3),
 (1, 0.5, 3.5),
 (1.5, 0.5, 3),
 (1.5, 0, 3.5),
 (3, 4, 1),
 (3.5, 4, 1.5),
 (4, 3, 0),
 (4, 3.5, 0.5),
 (2, 4, 0),
 (2.5, 4, 0.5),
 (0, 3, 3),
 (0.5, 3, 3.5),
 (0.5, 3.5, 3),
 (0, 3.5, 3.5),
 (1, 1, 0),
 (1, 1.5, 0.5),
 (1.5, 1, 0.5),
 (1.5, 1.5, 0),
 (2.5, 2.5, 1),
 (2, 2, 1),
 (2, 2.5, 1.5),
 (2.5, 2, 1.5),
 (4, 4, 3),
 (4, 2, 3),
 (4, 2.5, 3.5),
 (3, 0, 4),
 (3.5, 0.5, 4),
 (0, 2, 0),
 (0.5, 2, 0.5),
 (0.5, 2.5, 0),
 (0, 2.5, 0.5),
 (3, 1, 3),
 (3, 1.5, 3.5),
 (3.5, 1, 3.5),
 (3.5, 1.5, 3),
 (1, 2, 1),
 (1, 2.5, 1.5),
 (1.5, 2, 1.5),
 (1.5, 2.5, 1),
 (4, 1, 2),
 (4, 1.5, 2.5),
 (0, 4, 1),
 (0.5, 4, 1.5),
 (0, 1, 1),
 (0.5, 1, 1.5),
 (0.5, 1